In [64]:
import pandas as pd
import glob
import os

### 1. 지역 데이터 합치기

In [65]:
raw_dir = '../../data/1.raw/area'

# 폴더 안 CSV 전부 가져오기
csv_files = glob.glob(os.path.join(raw_dir, '*.csv'))

print("불러올 파일 개수:", len(csv_files))
for f in csv_files:
    print(os.path.basename(f))

# 파일 읽기
df_list = [pd.read_csv(f, encoding='cp949') for f in csv_files]

# 병합
merged_df = pd.concat(df_list, ignore_index=True)

print("병합 완료:", merged_df.shape)


불러올 파일 개수: 7
6_시군구_성_연령_5세_별_주민등록연앙인구_2023(충남,전남,전북).csv
3_시군구_성_연령_5세_별_주민등록연앙인구_2023_(세종시).csv
5_시군구_성_연령_5세_별_주민등록연앙인구_2023(강원충북).csv
4_시군구_성_연령_5세_별_주민등록연앙인구_2023_(경기_시만가져옴) 2.csv
7_시군구_성_연령_5세_별_주민등록연앙인구_2023(경남,경북,제주).csv
2_시군구_성_연령_5세_별_주민등록연앙인구_2023_(광주대전울산).csv
1_시군구_성_연령_5세_별_주민등록연앙인구_2023_(전국서울대구부산인천).csv
병합 완료: (786, 27)


### 2. 지역 별 행정구역(시) 컬럼 생성

In [66]:
# 타겟 시 리스트
target_values = ['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', 
                '광주광역시', '대전광역시', '울산광역시', 
                '세종특별자치시', 
                '경기도', 
                '강원도', '충청북도', 
                '충청남도', '전라북도', '전라남도', 
                '경상북도','경상남도', '제주특별자치도']

# 기존 컬럼명 저장
col = '행정구역(시군구)별'

# 새로운 컬럼 생성
merged_df['행정구역(시)'] = None

current_city = None

for idx, value in merged_df[col].items():
    if value in target_values:
        current_city = value  # 시 정보 업데이트
    merged_df.at[idx, '행정구역(시)'] = current_city


# 앞으로 배치하고 싶은 컬럼들
front_cols = ['시점', '행정구역(시)', '행정구역(시군구)별', '성별', '계']

# 나머지 컬럼 자동 추출
other_cols = [col for col in merged_df.columns if col not in front_cols]

# 순서 재정렬
merged_df = merged_df[front_cols + other_cols]

merged_df.head(10)

,시점,행정구역(시),행정구역(시군구)별,성별,계,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
0,2023,충청남도,충청남도,계,2119950.5,59200.5,88889.5,102580.0,99448.0,109083.0,...,140510.0,100723.0,75262.0,128397.0,68028.0,60369.0,42039.5,14917.5,2954.0,458.0
1,2023,충청남도,충청남도,남자,1084955.5,30359.5,45518.5,52955.0,51584.5,59384.0,...,70082.0,48421.0,34018.5,43389.5,25652.5,17737.0,13421.5,3682.5,575.5,57.5
2,2023,충청남도,충청남도,여자,1034995.0,28841.0,43371.0,49625.0,47863.5,49699.0,...,70428.0,52302.0,41243.5,85007.5,42375.5,42632.0,28618.0,11235.0,2378.5,400.5
3,2023,충청남도,천안시,계,654280.5,21233.5,30799.0,34455.5,33230.5,41974.0,...,30458.0,19151.5,13613.0,19848.5,10932.0,8916.5,6235.5,2144.0,462.5,74.5
4,2023,충청남도,천안시,남자,334803.5,10909.0,15784.0,17696.5,17193.0,21788.5,...,15193.0,9154.0,6108.5,6638.5,4121.5,2517.0,1935.5,488.5,85.0,8.0
5,2023,충청남도,천안시,여자,319477.0,10324.5,15015.0,16759.0,16037.5,20185.5,...,15265.0,9997.5,7504.5,13210.0,6810.5,6399.5,4300.0,1655.5,377.5,66.5
6,2023,충청남도,동남구,계,255159.5,6619.0,10182.5,12616.0,12853.5,16351.0,...,15119.5,9829.5,6958.5,10362.0,5632.5,4729.5,3291.5,1153.0,245.5,39.5
7,2023,충청남도,동남구,남자,129228.5,3395.0,5220.0,6503.5,6664.0,8533.5,...,7509.5,4729.5,3166.5,3529.5,2176.0,1353.5,1033.5,270.0,45.5,4.5
8,2023,충청남도,동남구,여자,125931.0,3224.0,4962.5,6112.5,6189.5,7817.5,...,7610.0,5100.0,3792.0,6832.5,3456.5,3376.0,2258.0,883.0,200.0,35.0
9,2023,충청남도,서북구,계,399121.0,14614.5,20616.5,21839.5,20377.0,25623.0,...,15338.5,9322.0,6654.5,9486.5,5299.5,4187.0,2944.0,991.0,217.0,35.0


### 3. 성별컬럼 생성 및 행 간소화

In [67]:
key_cols = ['시점', '행정구역(시)', '행정구역(시군구)별']

gender_pivot = merged_df.pivot(
    index=key_cols,
    columns='성별',
    values='계'   # 여기 '계'는 총인구 / 남자인구 / 여자인구 값
).reset_index()

gender_pivot = gender_pivot.rename(columns={
    '계': '총인구수',
    '남자': '남자인구수',
    '여자': '여자인구수'
})

In [68]:
age_df = merged_df[merged_df['성별'] == '계'].copy()

# 성별 컬럼은 필요 없어짐
age_df = age_df.drop(columns=['성별'])


In [69]:
final_df = pd.merge(
    gender_pivot,
    age_df,
    on=key_cols,
    how='left'
)


In [70]:
final_df = final_df.drop(columns=['계'])


In [71]:
print(final_df.shape)
final_df.head()


(262, 29)


,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
0,2023,강원도,강릉시,209691.5,103896.0,105795.5,4883.0,6892.0,8525.5,9143.5,...,16801.0,11195.5,9344.0,13191.0,7466.0,5725.0,3863.0,1452.0,353.5,56.5
1,2023,강원도,강원도,1526763.0,767650.0,759113.0,39645.5,53774.5,63967.0,66508.0,...,122388.5,75982.0,63703.5,95323.5,54346.0,40977.5,28362.5,9848.5,2410.5,356.0
2,2023,강원도,고성군,27219.5,14000.5,13219.0,557.5,727.5,771.0,821.0,...,2712.0,1764.5,1662.0,2487.5,1439.5,1048.0,719.5,240.5,76.0,12.0
3,2023,강원도,동해시,88678.5,45040.5,43638.0,2188.5,3438.5,4167.0,4184.5,...,6945.5,4622.0,3745.5,5043.0,2960.0,2083.0,1465.0,500.5,105.0,12.5
4,2023,강원도,삼척시,62897.0,31901.0,30996.0,1509.5,1784.0,2237.5,2669.0,...,5488.0,3934.0,3213.5,5083.0,2789.0,2294.0,1575.0,585.0,114.5,19.5


In [72]:
# 천안시 테스트
final_df[final_df['남자인구수'] == 334803.5]

,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
241,2023,충청남도,천안시,654280.5,334803.5,319477.0,21233.5,30799.0,34455.5,33230.5,...,30458.0,19151.5,13613.0,19848.5,10932.0,8916.5,6235.5,2144.0,462.5,74.5


In [73]:
# 동남구 테스트
final_df[final_df['여자인구수'] == 125931.0]

,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
233,2023,충청남도,동남구,255159.5,129228.5,125931.0,6619.0,10182.5,12616.0,12853.5,...,15119.5,9829.5,6958.5,10362.0,5632.5,4729.5,3291.5,1153.0,245.5,39.5


In [74]:
# 충첨남도 테스트 (18개)
final_df[final_df['행정구역(시)'] == '충청남도']

,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,0 - 4세,5 - 9세,10 - 14세,15 - 19세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
228,2023,충청남도,계룡시,45452.0,22670.5,22781.5,1333.0,2381.5,2956.5,3319.5,...,2389.5,1465.5,950.5,1434.0,751.0,683.0,474.0,168.5,34.5,6.0
229,2023,충청남도,공주시,102020.5,50890.5,51130.0,1845.5,2792.5,3650.5,4270.0,...,9351.0,6720.0,4866.5,8993.5,4700.5,4293.0,2936.5,1129.5,190.5,36.5
230,2023,충청남도,금산군,49945.0,25242.5,24702.5,754.5,1225.0,1943.0,2112.0,...,5164.5,3783.0,3005.0,5353.0,2971.0,2382.0,1680.5,569.5,112.0,20.0
231,2023,충청남도,논산시,111226.0,55530.0,55696.0,2248.5,3626.0,4343.0,4667.0,...,10047.5,7246.5,5591.5,10115.5,5290.5,4825.0,3336.5,1205.5,248.0,35.0
232,2023,충청남도,당진시,168800.0,89996.5,78803.5,5294.5,8019.5,8807.5,7506.5,...,10660.0,7831.5,5784.0,10258.0,5415.0,4843.0,3451.5,1129.5,224.0,38.0
233,2023,충청남도,동남구,255159.5,129228.5,125931.0,6619.0,10182.5,12616.0,12853.5,...,15119.5,9829.5,6958.5,10362.0,5632.5,4729.5,3291.5,1153.0,245.5,39.5
234,2023,충청남도,보령시,96232.0,48726.0,47506.0,1984.5,3019.0,3788.0,3928.0,...,8589.5,6531.5,4940.0,8040.5,4315.0,3725.5,2608.5,902.5,187.0,27.5
235,2023,충청남도,부여군,61562.5,30613.5,30949.0,779.5,1502.5,1941.5,2228.5,...,6743.5,5164.5,4024.0,7888.0,4034.0,3854.0,2640.0,968.0,218.5,27.5
236,2023,충청남도,서북구,399121.0,205575.0,193546.0,14614.5,20616.5,21839.5,20377.0,...,15338.5,9322.0,6654.5,9486.5,5299.5,4187.0,2944.0,991.0,217.0,35.0
237,2023,충청남도,서산시,175773.5,91797.5,83976.0,5403.0,7859.0,8834.0,8246.0,...,10964.5,8290.5,6036.0,10715.0,5497.0,5218.0,3580.0,1343.5,250.5,44.0


### 4. 연령 컬럼 그룹화

In [75]:
# 1. 연령 관련 컬럼들 자동으로 가져오기
#    (컬럼 이름에 '세'가 들어간 것들만)
age_cols = [c for c in final_df.columns if '세' in c]

print("연령 컬럼들:")
print(age_cols)

# 2. 연령대별로 묶을 구간 정하기
#    컬럼 순서를 기준으로 slicing 할 거라, 중간에 빠지는 거 없이 안전함

# 인덱스 찾기
idx_0_4   = age_cols.index('0 - 4세')
idx_5_9   = age_cols.index('5 - 9세')
idx_10_14 = age_cols.index('10 - 14세')
idx_15_19 = age_cols.index('15 - 19세')
idx_20_24 = age_cols.index('20 - 24세')
idx_35_39 = age_cols.index('35 - 39세')
idx_40_44 = age_cols.index('40 - 44세')
idx_55_59 = age_cols.index('55 - 59세')

# 각 연령대에 들어갈 컬럼 리스트
infant_cols   = [age_cols[idx_0_4]]                           # 영아기 0-4
child_cols    = [age_cols[idx_5_9]]                           # 아동기 5-9
teen_cols     = age_cols[idx_10_14 : idx_15_19 + 1]           # 청소년기 10-19
youth_cols    = age_cols[idx_20_24 : idx_35_39 + 1]           # 청년기 20-35
middle_cols   = age_cols[idx_40_44 : idx_55_59 + 1]           # 중년기 40-55
old_cols      = age_cols[idx_55_59 + 1 :]                     # 노년기 60세 이상 (60부터 끝까지)

print("청소년기:", teen_cols)
print("청년기:", youth_cols)
print("중년기:", middle_cols)
print("노년기:", old_cols)

# 3. 새 연령대 컬럼 만들기 (row별 합계)
final_df['영아기'] = final_df[infant_cols].sum(axis=1)
final_df['아동기'] = final_df[child_cols].sum(axis=1)
final_df['청소년기'] = final_df[teen_cols].sum(axis=1)
final_df['청년기'] = final_df[youth_cols].sum(axis=1)
final_df['중년기'] = final_df[middle_cols].sum(axis=1)
final_df['노년기'] = final_df[old_cols].sum(axis=1)

# 4. 기존 연령 관련 컬럼들은 드롭
final_df = final_df.drop(columns=age_cols)


연령 컬럼들:
['0 - 4세', '5 - 9세', '10 - 14세', '15 - 19세', '20 - 24세', '25 - 29세', '30 - 34세', '35 - 39세', '40 - 44세', '45 - 49세', '50 - 54세', '55 - 59세', '60 - 64세', '65 - 69세', '70 - 74세', '75 - 79세', '80세 이상', '80 - 84세', '85세 이상', '85 - 89세', '90 - 94세', '95 - 99세', '100세 이상']
청소년기: ['10 - 14세', '15 - 19세']
청년기: ['20 - 24세', '25 - 29세', '30 - 34세', '35 - 39세']
중년기: ['40 - 44세', '45 - 49세', '50 - 54세', '55 - 59세']
노년기: ['60 - 64세', '65 - 69세', '70 - 74세', '75 - 79세', '80세 이상', '80 - 84세', '85세 이상', '85 - 89세', '90 - 94세', '95 - 99세', '100세 이상']


In [80]:
# 결과 확인
final_df.head()

,시점,행정구역(시),행정구역(시군구)별,총인구수,남자인구수,여자인구수,영아기,아동기,청소년기,청년기,중년기,노년기
0,2023,강원도,강릉시,209691.5,103896.0,105795.5,4883.0,6892.0,17669.0,43594.5,66246.0,89323.0
1,2023,강원도,강원도,1526763.0,767650.0,759113.0,39645.5,53774.5,130475.0,325374.0,471897.5,641897.5
2,2023,강원도,고성군,27219.5,14000.5,13219.0,557.5,727.5,1592.0,4891.0,7642.5,15344.5
3,2023,강원도,동해시,88678.5,45040.5,43638.0,2188.5,3438.5,8351.5,16985.5,29171.0,35669.5
4,2023,강원도,삼척시,62897.0,31901.0,30996.0,1509.5,1784.0,4906.5,11630.5,18733.0,31710.5
